In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [19]:
df = pd.read_csv("datasets/earLobe.csv")

In [20]:
df.head()

,patientID,has_DM2,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,...,Var3152,Var3153,Var3154,Var3155,Var3156,Var3157,Var3158,Var3159,Var3160,Var3161
0,ramanShift,NaN,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,...,3150,3151,3152,3153,3154,3155,3156,3157,3158,3159
1,DM201,1.0,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,...,0,0,0,0,0,0,0,0,0,0
2,DM202,1.0,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,...,0,0,0,0,0,0,0,0,0,0
3,DM203,1.0,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,...,0,0,0,0,0,0,0,0,0,0
4,DM204,1.0,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,...,0,0,0,0,0,0,0,0,0,0


Вырезаем колонку с 'patientID'

In [22]:
patientID = df.pop('patientID')

Удаляем строку с частотами

In [28]:
df.drop(0, inplace=True)

Разделяем данные на матрицу с признаками X и на столбец с целевой переменной (метками)

In [30]:
X, y = df.drop('has_DM2', axis=1), df['has_DM2']

In [32]:
X.shape, y.shape

((20, 3160), (20,))

In [33]:
from sklearn.model_selection import train_test_split

In [70]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=.3, random_state=7)

In [71]:
X_train.shape, X_holdout.shape

((14, 3160), (6, 3160))

In [72]:
from sklearn.tree import DecisionTreeClassifier

In [73]:
tree = DecisionTreeClassifier(random_state=17)

fit-predict

In [74]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [75]:
from sklearn.metrics import accuracy_score

In [76]:
pred_holdout = tree.predict(X_holdout)

In [77]:
pred_holdout.shape, y_holdout.shape

((6,), (6,))

In [78]:
accuracy_score(y_holdout, pred_holdout)

0.6666666666666666

In [110]:
y.value_counts(normalize=True)

1.0    0.55
0.0    0.45
Name: has_DM2, dtype: float64

#### Точность предсказания должна быть лучше как минимум чем 55%

In [80]:
from sklearn.linear_model import LogisticRegression

In [90]:
logreg = LogisticRegression(random_state=17)
logreg.fit(X_train, y_train);

/home/andrey/Projects/Kaggle-Raman-ML/env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [91]:
pred_holdout_logreg = logreg.predict(X_holdout)
accuracy_score(y_holdout, pred_holdout_logreg)

0.6666666666666666

In [93]:
#y_train, y_holdout

In [101]:
from sklearn.ensemble import RandomForestClassifier

In [106]:
clf = RandomForestClassifier(n_jobs=2, random_state=7)
clf.fit(X_train, y_train);

/home/andrey/Projects/Kaggle-Raman-ML/env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [107]:
pred_holdout_clf = clf.predict(X_holdout)
accuracy_score(y_holdout, pred_holdout_clf)

0.6666666666666666

In [111]:
from xgboost import XGBClassifier

In [114]:
xgbc = XGBClassifier()
xgbc.fit(X_train, y_train);

In [115]:
pred_holdout_xgbc = xgbc.predict(X_holdout)
accuracy_score(y_holdout, pred_holdout_xgbc)

0.8333333333333334

In [116]:
from sklearn import svm

In [119]:
clf_svm = svm.SVC(gamma='scale')
clf_svm.fit(X_train, y_train);

In [120]:
pred_holdout_svm = clf_svm.predict(X_holdout)
accuracy_score(y_holdout, pred_holdout_svm)

0.6666666666666666

In [134]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

#### Кросс-валидация

In [135]:
skf = StratifiedKFold(n_splits = 5,shuffle = True)
cross_val_score(clf_svm, X, y, cv=skf, scoring='accuracy')

array([0.4       , 0.25      , 0.5       , 0.5       , 0.66666667])

In [136]:
cross_val_score(clf, X, y, cv=skf, scoring='accuracy')

array([0.6       , 0.5       , 0.75      , 0.75      , 0.66666667])